#Use a proper tensorflow version
Our model was build on TF2.14 and it does cause some compatibility issues with TF2.17 so make sure you use the right version

In [ ]:
#Check tf version
import tensorflow as tf
#print(tf.__version__)

#The following force you to use tensorflow 2.14 but you need to restart. You may comment it if you know you are using 2.14
##Make sure you down grade!! otherwise, some part will fail due to compatibility.
#!pip3 install --upgrade tensorflow==2.14.0
print(tf.__version__)

2.17.0


#Get the dataset

In [ ]:
# mount google drive on your runtime using and authorization code.
# more details here: https://colab.research.google.com/notebooks/io.ipynb
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Set directories correctly

In [ ]:
import os
# Path to the directory containing the project files (CHANGE THIS PATH TO THE DIRECTORY ON YOUR COMPUTER)
PROJECT_ROOT_DIR = os.getcwd() + '/'

# Path to the directory containing the dataset relative to project file
DATA_DIR = 'drive/My Drive/Colab Notebooks/GTSRB_dataset/GTSRB_dataset/'

#path to the directory you want to use for saving models relative to the project file
MODEL_DIR = 'drive/My Drive/Colab Notebooks/'

In [ ]:
# Funciton for loading the dataset
# Code from advml-traffic-sign (https://github.com/inspire-group/advml-traffic-sign)
def load_dataset_GTSRB(n_channel=3, train_file_name=None):
    """
    Load GTSRB data as a (datasize) x (channels) x (height) x (width) numpy
    matrix. Each pixel is rescaled to the range [0,1].
    """

    def load_pickled_data(file, columns):
        """
        Loads pickled training and test data.

        Parameters
        ----------
        file    : string
                          Name of the pickle file.
        columns : list of strings
                          List of columns in pickled data we're interested in.

        Returns
        -------
        A tuple of datasets for given columns.
        """

        with open(file, mode='rb') as f:
            dataset = pickle.load(f)
        return tuple(map(lambda c: dataset[c], columns))

    def preprocess(x, n_channel):
        """
        Preprocess dataset: turn images into grayscale if specified, normalize
        input space to [0,1], reshape array to appropriate shape for NN model
        """

        if n_channel == 3:
            # Scale features to be in [0, 1]
            x = (x / 255.).astype(np.float32)
        else:
            # Convert to grayscale, e.g. single Y channel
            x = 0.299 * x[:, :, :, 0] + 0.587 * x[:, :, :, 1] + \
                0.114 * x[:, :, :, 2]
            # Scale features to be in [0, 1]
            x = (x / 255.).astype(np.float32)
            x = x[:, :, :, np.newaxis]
        return x

    # Load pickle dataset
    if train_file_name is None:
        x_train, y_train = load_pickled_data(
            PROJECT_ROOT_DIR + DATA_DIR + 'train.p', ['features', 'labels'])
    else:
        x_train, y_train = load_pickled_data(
            PROJECT_ROOT_DIR + DATA_DIR + train_file_name, ['features', 'labels'])
    x_val, y_val = load_pickled_data(
        PROJECT_ROOT_DIR + DATA_DIR + 'valid.p', ['features', 'labels'])
    x_test, y_test = load_pickled_data(
        PROJECT_ROOT_DIR + DATA_DIR + 'test.p', ['features', 'labels'])

    # Preprocess loaded data
    x_train = preprocess(x_train, n_channel)
    x_val = preprocess(x_val, n_channel)
    x_test = preprocess(x_test, n_channel)
    return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
import pickle
import numpy as np
# Load the images and labels. These images are RGB so we have 3 channels
imgs_train, labels_train, imgs_val, labels_val, imgs_test, labels_test = load_dataset_GTSRB(n_channel=3)

#Load the built model from Assignment 1 (Already on blackboard)

In [ ]:
## Load the saved the model
from tensorflow.keras.models import load_model



#load our already trained model
# Your earlier directories should change to match this one
#You should be able to load the model once trained. The following line is used for that.
model = load_model(MODEL_DIR +'VGG_best(1).h5')

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


#Some constants that you might need
It is not neccessary that you will use all

In [ ]:
# Set constants (GTSRB)
NUM_LABELS = 43                             # Number of labels or classes for classification
BATCH_SIZE = 128                            # Size of batch
HEIGHT = 32                                 # Height of input image
WIDTH = 32                                  # Width of input image
N_CHANNEL = 3                               # Number of channels
OUTPUT_DIM = 43                             # Number of output dimension

# Set training hyperparameters
NUM_EPOCH = 50                            # Number of epoch to train
LR = 0.0002                                 # Learning rate
RBW = True #restore best weights
PATIENCE = 5# how many epochs between improvements
sample_size = 100
alpha = 0.001
INPUT_SHAPE = (HEIGHT, WIDTH, N_CHANNEL)  # Input shape of model
IMG_SHAPE = (HEIGHT, WIDTH, N_CHANNEL)

# setting up labels

In [ ]:

from tensorflow.keras.utils import to_categorical

#Convert the labels to one-hot encoding (to input to the models)

#Use the following sub-test dataset for your certified training
imgs_sub_test = imgs_test[0:500,:,:,:].copy()
labels_adv = labels_test[0:500].copy()




labels_train_cat = to_categorical(labels_train, NUM_LABELS)
labels_test_cat = to_categorical(labels_test, NUM_LABELS)
labels_val_cat = to_categorical(labels_val, NUM_LABELS)

#for testing adversarial inputs
labels_adv_cat = to_categorical(labels_adv,NUM_LABELS)

print('Labels train shape: {}'.format(labels_train.shape))
print('Labels train catagorical shape: {}\n'.format(labels_train_cat.shape))
print('Labels Adver shape: {}'.format(labels_adv.shape))
print('Labels Adver catagorical shape: {}'.format(labels_adv_cat.shape))

print('Imgs Adver shape: {}'.format(imgs_sub_test.shape))

Labels train shape: (34799,)
Labels train catagorical shape: (34799, 43)

Labels Adver shape: (500,)
Labels Adver catagorical shape: (500, 43)
Imgs Adver shape: (500, 32, 32, 3)


#Install ART if needed

In [ ]:
#Install ART
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.9 MB/s eta 0:00:00


#Some imports.
You might need more

In [ ]:
from art.estimators.classification import TensorFlowV2Classifier

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

#Task 4

We will leave the details of implimentation to you

Now, let's defend test defense with DP. We have built a DP-SGD model using the TensorFlow privacy library. The model is given to you, and you just need to analyze it. First, check the overall DP code and answer the below questions. Then, analyze the given DP model in terms of accuracy and resiliency to membership inference attacks. Use a label-only membership inference attack with 5 max-queries and the same setup as in Task 2.


2.	[5 pts] Measure the differential privacy guarantee of the given model.

3.	[5 pts] Calculate the model's performance under attacks and any other observations regarding the results.


In [ ]:
!pip install tensorflow_privacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.24.0
    Uninstalling tensorflow-probability-0.24.0:
      Successfully uninstalled tensorflow-probability-0.24.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.13 requires packaging<25,>=23

In [ ]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy_lib
sample_size = len(imgs_train)
delta = 1e-5
epsilon, _ = compute_dp_sgd_privacy_lib.compute_dp_sgd_privacy(
    n=sample_size,
    batch_size=BATCH_SIZE,
    noise_multiplier=1,
    epochs=NUM_EPOCH,
    delta=delta
)

print(f"(ε, δ)-DP: ({epsilon:.2f}, {delta})")

(ε, δ)-DP: (2.56, 1e-05)


In [ ]:
##Some variables to set  number of samples used to calibrate distance threshold
model.load_weights(MODEL_DIR +'dp_model_weights.h5')
x = np.concatenate([imgs_train, imgs_test]) #Merge train and test dataset
y = np.concatenate([labels_train_cat, labels_test_cat]) #Merge train and test dataset
training_sample = np.array([1] * len(imgs_train) + [0] * len(imgs_test)) #Class label for the attack model training. What does it say?

from numpy.random import choice

##Random samples to be used for training our attack model
# evaluation data
n = 500
eval_data_idx = choice(len(x), n) #choose 1500 of the samples
x_eval, y_eval = x[eval_data_idx], y[eval_data_idx] #creating your training dataset for the attack model
eval_label = training_sample[eval_data_idx] #creating your training label for the attack model

In [ ]:

import torch.optim as optim
from art.attacks.inference.membership_inference import LabelOnlyDecisionBoundary
from sklearn.metrics import accuracy_score
from art.estimators.classification.pytorch import PyTorchClassifier
from torch import nn
# criterion = nn.CrossEntropyLoss()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
art_model = TensorFlowV2Classifier(
    model=model,
    nb_classes=NUM_LABELS,
    input_shape=INPUT_SHAPE,
    loss_object=loss_object,
    optimizer=optimizer,
    clip_values=(0, 1)
)

# Members and Non-members accuracy at queries = 5
mia_label_only_unsupervised = LabelOnlyDecisionBoundary(art_model)
mia_label_only_unsupervised.calibrate_distance_threshold_unsupervised(top_t=20, num_samples=200, max_queries=5, verbose=True, batch_size=256)

HopSkipJump:   0%|          | 0/200 [00:00<?, ?it/s]

HopSkipJump:   0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/art/attacks/evasion/hop_skip_jump.py:606: RuntimeWarning: invalid value encountered in divide
  rnd_noise = (eval_samples - current_sample) / delta


HopSkipJump:   0%|          | 0/6 [00:00<?, ?it/s]

HopSkipJump:   0%|          | 0/6 [00:00<?, ?it/s]

HopSkipJump:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
pred_label_unsupervised = mia_label_only_unsupervised.infer(x_eval, y_eval)

HopSkipJump:   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
members_accuracy = accuracy_score(eval_label[eval_label == 1], pred_label_unsupervised[eval_label == 1])
non_members_accuracy = accuracy_score(eval_label[eval_label == 0], pred_label_unsupervised[eval_label == 0])
overall_accuracy = accuracy_score(eval_label, pred_label_unsupervised)
print("Members Accuracy: %f" % members_accuracy)
print("Non Members Accuracy: %f" % non_members_accuracy)
print("Overall Accuracy: %f" % overall_accuracy)

Members Accuracy: 0.317935
Non Members Accuracy: 0.712121
Overall Accuracy: 0.422000
